In [10]:
import websocket, json, pprint, talib, numpy
from binance.client import Client
from binance.enums import *
import ../config.py

SyntaxError: invalid syntax (3765565360.py, line 4)

In [3]:
SOCKET = "wss://stream.binance.com:9443/ws/ethusdt@kline_1m"

RSI_PERIOD = 14
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 30
TRADE_SYMBOL = 'ETHUSD'
TRADE_QUANTITY = 0.05

closes = []
in_position = False

client = Client("", "dQVB0zrnaO8rNNss5oD0vbC7EkgKr720rGHTtOqjeU5b1OE6dcv93aP4btQWYLGH", tld='us')

def order(side, quantity, symbol,order_type=ORDER_TYPE_MARKET):
    try:
        print("sending order")
        order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
        print(order)
    except Exception as e:
        print("an exception occured - {}".format(e))
        return False

    return True

    
def on_open(ws):
    print('opened connection')

def on_close(ws):
    print('closed connection')

def on_message(ws, message):
    global closes, in_position
    
    print('received message')
    json_message = json.loads(message)
    pprint.pprint(json_message)

#     candle = json_message['k']

#     is_candle_closed = candle['x']
#     close = candle['c']

#     if is_candle_closed:
#         print("candle closed at {}".format(close))
#         closes.append(float(close))
#         print("closes")
#         print(closes)

#         if len(closes) > RSI_PERIOD:
#             np_closes = numpy.array(closes)
#             rsi = talib.RSI(np_closes, RSI_PERIOD)
#             print("all rsis calculated so far")
#             print(rsi)
#             last_rsi = rsi[-1]
#             print("the current rsi is {}".format(last_rsi))

#             if last_rsi > RSI_OVERBOUGHT:
#                 if in_position:
#                     print("Overbought! Sell! Sell! Sell!")
#                     # put binance sell logic here
#                     order_succeeded = order(SIDE_SELL, TRADE_QUANTITY, TRADE_SYMBOL)
#                     if order_succeeded:
#                         in_position = False
#                 else:
#                     print("It is overbought, but we don't own any. Nothing to do.")
            
#             if last_rsi < RSI_OVERSOLD:
#                 if in_position:
#                     print("It is oversold, but you already own it, nothing to do.")
#                 else:
#                     print("Oversold! Buy! Buy! Buy!")
#                     # put binance buy order logic here
#                     order_succeeded = order(SIDE_BUY, TRADE_QUANTITY, TRADE_SYMBOL)
#                     if order_succeeded:
#                         in_position = True

                
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()

opened connection
received message
{'E': 1648241035786,
 'e': 'kline',
 'k': {'B': '0',
       'L': 789718244,
       'Q': '192780.83748100',
       'T': 1648241039999,
       'V': '61.75000000',
       'c': '3123.31000000',
       'f': 789717932,
       'h': '3123.32000000',
       'i': '1m',
       'l': '3120.98000000',
       'n': 313,
       'o': '3121.62000000',
       'q': '349539.23306600',
       's': 'ETHUSDT',
       't': 1648240980000,
       'v': '111.95160000',
       'x': False},
 's': 'ETHUSDT'}
received message
{'E': 1648241038330,
 'e': 'kline',
 'k': {'B': '0',
       'L': 789718251,
       'Q': '198264.14421900',
       'T': 1648241039999,
       'V': '63.50560000',
       'c': '3123.48000000',
       'f': 789717932,
       'h': '3123.48000000',
       'i': '1m',
       'l': '3120.98000000',
       'n': 320,
       'o': '3121.62000000',
       'q': '355187.45786800',
       's': 'ETHUSDT',
       't': 1648240980000,
       'v': '113.76000000',
       'x': False},
 's

received message
{'E': 1648241073023,
 'e': 'kline',
 'k': {'B': '0',
       'L': 789718652,
       'Q': '328864.28262000',
       'T': 1648241099999,
       'V': '105.27870000',
       'c': '3123.98000000',
       'f': 789718283,
       'h': '3124.85000000',
       'i': '1m',
       'l': '3121.82000000',
       'n': 370,
       'o': '3123.80000000',
       'q': '568259.78987300',
       's': 'ETHUSDT',
       't': 1648241040000,
       'v': '181.91980000',
       'x': False},
 's': 'ETHUSDT'}
received message
{'E': 1648241075357,
 'e': 'kline',
 'k': {'B': '0',
       'L': 789718661,
       'Q': '330479.23661700',
       'T': 1648241099999,
       'V': '105.79560000',
       'c': '3124.32000000',
       'f': 789718283,
       'h': '3124.85000000',
       'i': '1m',
       'l': '3121.82000000',
       'n': 379,
       'o': '3123.80000000',
       'q': '572303.59023800',
       's': 'ETHUSDT',
       't': 1648241040000,
       'v': '183.21410000',
       'x': False},
 's': 'ETHUSDT'}
re

received message
{'E': 1648241112069,
 'e': 'kline',
 'k': {'B': '0',
       'L': 789718993,
       'Q': '31539.95470100',
       'T': 1648241159999,
       'V': '10.09340000',
       'c': '3124.13000000',
       'f': 789718867,
       'h': '3125.41000000',
       'i': '1m',
       'l': '3124.13000000',
       'n': 127,
       'o': '3125.09000000',
       'q': '85321.42134900',
       's': 'ETHUSDT',
       't': 1648241100000,
       'v': '27.30240000',
       'x': False},
 's': 'ETHUSDT'}
received message
{'E': 1648241114527,
 'e': 'kline',
 'k': {'B': '0',
       'L': 789719001,
       'Q': '39642.09937700',
       'T': 1648241159999,
       'V': '12.68680000',
       'c': '3124.14000000',
       'f': 789718867,
       'h': '3125.41000000',
       'i': '1m',
       'l': '3124.13000000',
       'n': 135,
       'o': '3125.09000000',
       'q': '93623.82275800',
       's': 'ETHUSDT',
       't': 1648241100000,
       'v': '29.95990000',
       'x': False},
 's': 'ETHUSDT'}
received m

received message
{'E': 1648241152342,
 'e': 'kline',
 'k': {'B': '0',
       'L': 789719226,
       'Q': '139146.66345800',
       'T': 1648241159999,
       'V': '44.55610000',
       'c': '3121.09000000',
       'f': 789718867,
       'h': '3125.41000000',
       'i': '1m',
       'l': '3120.72000000',
       'n': 360,
       'o': '3125.09000000',
       'q': '345134.36923100',
       's': 'ETHUSDT',
       't': 1648241100000,
       'v': '110.51430000',
       'x': False},
 's': 'ETHUSDT'}
received message
{'E': 1648241154512,
 'e': 'kline',
 'k': {'B': '0',
       'L': 789719233,
       'Q': '144110.46089800',
       'T': 1648241159999,
       'V': '46.14650000',
       'c': '3121.09000000',
       'f': 789718867,
       'h': '3125.41000000',
       'i': '1m',
       'l': '3120.72000000',
       'n': 367,
       'o': '3125.09000000',
       'q': '354884.98240400',
       's': 'ETHUSDT',
       't': 1648241100000,
       'v': '113.63840000',
       'x': False},
 's': 'ETHUSDT'}
rece

received message
{'E': 1648241193175,
 'e': 'kline',
 'k': {'B': '0',
       'L': 789719386,
       'Q': '15078.63601900',
       'T': 1648241219999,
       'V': '4.83160000',
       'c': '3120.88000000',
       'f': 789719259,
       'h': '3121.17000000',
       'i': '1m',
       'l': '3120.73000000',
       'n': 128,
       'o': '3121.16000000',
       'q': '203877.60760900',
       's': 'ETHUSDT',
       't': 1648241160000,
       'v': '65.32730000',
       'x': False},
 's': 'ETHUSDT'}
received message
{'E': 1648241195241,
 'e': 'kline',
 'k': {'B': '0',
       'L': 789719392,
       'Q': '22145.52631900',
       'T': 1648241219999,
       'V': '7.09590000',
       'c': '3121.01000000',
       'f': 789719259,
       'h': '3121.17000000',
       'i': '1m',
       'l': '3120.73000000',
       'n': 134,
       'o': '3121.16000000',
       'q': '210944.49790900',
       's': 'ETHUSDT',
       't': 1648241160000,
       'v': '67.59160000',
       'x': False},
 's': 'ETHUSDT'}
received m

received message
{'E': 1648241232446,
 'e': 'kline',
 'k': {'B': '0',
       'L': 789719538,
       'Q': '5964.77218200',
       'T': 1648241279999,
       'V': '1.91140000',
       'c': '3120.63000000',
       'f': 789719495,
       'h': '3120.63000000',
       'i': '1m',
       'l': '3120.62000000',
       'n': 44,
       'o': '3120.63000000',
       'q': '7120.33776800',
       's': 'ETHUSDT',
       't': 1648241220000,
       'v': '2.28170000',
       'x': False},
 's': 'ETHUSDT'}
received message
{'E': 1648241234797,
 'e': 'kline',
 'k': {'B': '0',
       'L': 789719541,
       'Q': '6186.96103800',
       'T': 1648241279999,
       'V': '1.98260000',
       'c': '3120.62000000',
       'f': 789719495,
       'h': '3120.63000000',
       'i': '1m',
       'l': '3120.62000000',
       'n': 47,
       'o': '3120.63000000',
       'q': '7385.27911800',
       's': 'ETHUSDT',
       't': 1648241220000,
       'v': '2.36660000',
       'x': False},
 's': 'ETHUSDT'}
received message
{'E

received message
{'E': 1648241273407,
 'e': 'kline',
 'k': {'B': '0',
       'L': 789719679,
       'Q': '53384.08541500',
       'T': 1648241279999,
       'V': '17.10470000',
       'c': '3121.54000000',
       'f': 789719495,
       'h': '3122.19000000',
       'i': '1m',
       'l': '3120.01000000',
       'n': 185,
       'o': '3120.63000000',
       'q': '117981.39721800',
       's': 'ETHUSDT',
       't': 1648241220000,
       'v': '37.79830000',
       'x': False},
 's': 'ETHUSDT'}
received message
{'E': 1648241275592,
 'e': 'kline',
 'k': {'B': '0',
       'L': 789719681,
       'Q': '53679.07189000',
       'T': 1648241279999,
       'V': '17.19920000',
       'c': '3121.55000000',
       'f': 789719495,
       'h': '3122.19000000',
       'i': '1m',
       'l': '3120.01000000',
       'n': 187,
       'o': '3120.63000000',
       'q': '118276.38369300',
       's': 'ETHUSDT',
       't': 1648241220000,
       'v': '37.89280000',
       'x': False},
 's': 'ETHUSDT'}


False